# note: This program needs openAI API

# Prep dataset

In [1]:
import pandas as pd
import numpy as np
import os
import scipy
from torch import tensor

os.getcwd()

'/Users/yancong/Desktop/4 clinical/02 projects_parsely/05 ssd-lm-stanglab/15 simulation/GPT3simul'

In [2]:
# reading in data by sample
# each sample is collapsed from the two open-ended prompts

data_foloder = '/Gradient_shuffle/'
result = '/Analysis/'
baseline = pd.read_csv(data_foloder + 'simulation_HV_baseline_vb_response_deid_v3.csv')
incoh10 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_10v3.csv')
incoh20 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_20v3.csv')
incoh50 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_50v3.csv')
ineff10 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_10v3.csv')
ineff20 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_20v3.csv')
ineff50 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_50v3.csv')

baseline.head() 

,grid,content,n_words
0,10455,"I'm a young man , an en an en- an engineer by ...",421
1,11689,Sure . I'm thirty three years old . My name is...,159
2,12376,Alright . um I live in not especially cool Spr...,468
3,12630,um So I'm currently twenty-nine . I was born a...,966
4,13493,Mhm . I'm a thirty five year old man who uh um...,134


# Install lib and dependencies

In [3]:
# no need to install it every time you open it
# for GPT-3
# install GPT-3 from huggingface
!pip install transformers 

You should consider upgrading via the '/Users/yancong/opt/anaconda3/envs/nlum1/bin/python -m pip install --upgrade pip' command.


In [4]:
# import tokenizers from GPT2TokenizerFast (GPT-3 uses the same tokenizer)

import transformers
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [6]:
# use openai API
from openai.embeddings_utils import get_embeddings, cosine_similarity # important
import openai
openai.api_key = 'your key'

# Get word embeddings from whole response embeddings GPT3 3df

In [7]:
buglist = {} # initialize dic to log bugs
def calc_gpt3(r): # defining function for extracting contextualized embeddings from verbatim transcripts
  if r[0] % 2 != 0: # keeping track of progress - print out even grid as we process them
    print('current grid: ', r[0])
  if type(r['content']) == str and r['n_words'] < 2048: # if the content is string [non-empty]; 2048 is max for GPT-3
    temp = r['content'].split(' ') # gives a list of words
    input = ' '.join([i for i in temp if (i != '') & ('{' not in i) & ('}' not in i) & ('#' not in i)]) # exclude NSV and the restart symbol
    try:
      vec = get_embeddings(input.split(' '), engine = 'text-similarity-babbage-001') # take in the whole response as a list of words
    except:
      buglist[r[0]] = input.split(' ') # store grid if there is a bug
    return vec
  else:
    return np.nan

# double check your input df
# make sure there is no '  ' or '   '
# put together all the data frames into 1 list
# to avoid repeated work
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  df["gpt3_embed"] = ''
  df["gpt3_embed"] = df.apply(lambda r: calc_gpt3(r), axis = 1) 
  df.to_csv(result + str(temp) + '.csv')

df.head()

current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493
current grid:  10455
current grid:  11689
current grid:  13493


,grid,content,n_words,gpt3_embed
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[0.009009403176605701, -0.017146145924925804,..."
1,11689,We have been using that opportunity to do more...,159,"[[-0.00924572255462408, -0.014978389255702496,..."
2,12376,Alright . um I live in not especially cool Spr...,468,"[[0.010747547261416912, -0.01882629469037056, ..."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[-0.012155817821621895, -0.022000038996338844..."
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[0.003884142730385065, -0.01114651095122099, ..."


In [8]:
buglist

{}

# MV 5/10

In [9]:
# Average semantic similarity of each word in 5- or 10- words window

def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
# n: How many elements each
# list should have
test = ['1','5','99','34','109','gh','io','wer','90','901','98','iop','er4','op0']
divide_chunks(test,5)
print(list(divide_chunks(test,5)))

[['1', '5', '99', '34', '109'], ['gh', 'io', 'wer', '90', '901'], ['98', 'iop', 'er4', 'op0']]


In [12]:
temp_file = -1 # keep track of intermediate output files names

for df in dfs:
    temp_file += 1 
    
    df['similarity'] = '' # create new empty columns
    df['gpt3_word_mv5'] = ''
    df['gpt3_word_mv10'] = ''

    ks = ['5', '10']
    for k in ks:
        # print progress
        cur = 'gpt3_word_mv' + k 
        print('current: ', cur)
        # loop over each response
        for i in df.index:
            if type(df['gpt3_embed'][i]) != float: 
                # chop 1 big response sequence into 5/10-token chunks
                word_embed_chunk = list(divide_chunks(df['gpt3_embed'][i], int(k)))
                chunk_temp_sum = [] 
                # loop over each chunk in the response
                for chunck_id, word_embed in enumerate(word_embed_chunk):
                    temp_sum = []
                    # calculate average similarities for that chunk
                    for word_id, embed in enumerate(word_embed):
                        w1 = embed
                        try:
                            w2 = word_embed[word_id+1]
                        except IndexError:
                            continue
                        temp = cosine_similarity(w1, w2)
                        temp_sum.append(temp)
                    temp_sim = np.nanmean(temp_sum)
                    chunk_temp_sum.append(temp_sim) # incrementally append similarity mean to the list 

            # get a list of similarity means for that response, 
            # its len is the number of chunks that the response can be chopped into
            df['similarity'][i] = chunk_temp_sum 

            # add other stats here
            sim = np.nanmean(chunk_temp_sum)
            df[cur][i] = sim
    df.to_csv(result + str(temp_file) + '.csv')
df.head()

current:  gpt3_word_mv5
current:  gpt3_word_mv10


/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_45904/3508527720.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'][i] = chunk_temp_sum
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_45904/3508527720.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cur][i] = sim
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_45904/3508527720.py:28: RuntimeWarning: Mean of empty slice
  temp_sim = np.nanmean(temp_sum)


current:  gpt3_word_mv5
current:  gpt3_word_mv10
current:  gpt3_word_mv5
current:  gpt3_word_mv10
current:  gpt3_word_mv5
current:  gpt3_word_mv10
current:  gpt3_word_mv5
current:  gpt3_word_mv10
current:  gpt3_word_mv5
current:  gpt3_word_mv10
current:  gpt3_word_mv5
current:  gpt3_word_mv10


,grid,content,n_words,gpt3_embed,similarity,gpt3_word_mv5,gpt3_word_mv10
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[0.009009403176605701, -0.017146145924925804,...","[0.8556107616829599, 0.7243433395212227, 0.780...",0.776858,0.777624
1,11689,We have been using that opportunity to do more...,159,"[[-0.00924572255462408, -0.014978389255702496,...","[0.8087813300097076, 0.8080145213914288, 0.788...",0.786263,0.783373
2,12376,Alright . um I live in not especially cool Spr...,468,"[[0.010747547261416912, -0.01882629469037056, ...","[0.7559491116579808, 0.7876836248597551, 0.776...",0.784385,0.785468
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[-0.012155817821621895, -0.022000038996338844...","[0.7832724098667, 0.8118012931973864, 0.756688...",0.787379,0.787312
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[0.003884142730385065, -0.01114651095122099, ...","[0.8039042835152086, 0.7656657122352821, 0.743...",0.779932,0.780051


In [14]:
temp = -1
for df in dfs:
    temp += 1
    # get rid of the embeddings columns
    # because it's too big
    df = df[['grid',
       'gpt3_word_mv5', 'gpt3_word_mv10']]
    df['grid'] = df['grid'].astype(str)
    df.to_csv(result + str(temp) + '.csv')
df.head()

/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_45904/463216634.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grid'] = df['grid'].astype(str)


,grid,gpt3_word_mv5,gpt3_word_mv10
0,10455,0.776858,0.777624
1,11689,0.786263,0.783373
2,12376,0.784385,0.785468
3,12630,0.787379,0.787312
4,13493,0.779932,0.780051


# K2:10

In [12]:
import ast # a module that evaluates mathematical expressions and statements

# this might take a while
# do not run 2 scripts at once, when using vscode
temp = -1
ks=['2', '3', '4', '5', '6', '7', '8', '9', '10']

for df in dfs:
    temp += 1
    # create new empty columns
    df['gpt3_word_k2'] = ''
    df['gpt3_word_k3'] = ''
    df['gpt3_word_k4'] = ''
    df['gpt3_word_k5'] = ''
    df['gpt3_word_k6'] = ''
    df['gpt3_word_k7'] = ''
    df['gpt3_word_k8'] = ''
    df['gpt3_word_k9'] = ''
    df['gpt3_word_k10'] = ''
    df.to_csv(result + str(temp) + '.csv')
df.head()

,grid,content,n_words,gpt3_embed,gpt3_word_mv5,gpt3_word_mv10,gpt3_word_k2,gpt3_word_k3,gpt3_word_k4,gpt3_word_k5,gpt3_word_k6,gpt3_word_k7,gpt3_word_k8,gpt3_word_k9,gpt3_word_k10
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[0.009009403176605701, -0.017146145924925804,...",0.776863,0.777622,,,,,,,,,
1,11689,We have been using that opportunity to do more...,159,"[[-0.00924572255462408, -0.014978389255702496,...",0.786322,0.783411,,,,,,,,,
2,12376,Alright . um I live in not especially cool Spr...,468,"[[0.010747547261416912, -0.01882629469037056, ...",0.784362,0.785444,,,,,,,,,
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[-0.012155817821621895, -0.022000038996338844...",NaN,NaN,,,,,,,,,
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[0.003884142730385065, -0.01114651095122099, ...",NaN,NaN,,,,,,,,,


In [14]:
temp_file = -1
ks=['2', '3', '4', '5', '6', '7', '8', '9', '10']

for df in dfs:
    temp_file += 1
    # loop through each k
    for k in ks:
        cur = 'gpt3_word_k' + k 
        print('Coherence k ', k)
        # loop through each response
        for i in df.index:
            if type(df['gpt3_embed'][i]) != float:
                temp = []
                # calcuate similarity of word pairs at k inter-token distance
                for id,v in enumerate(df['gpt3_embed'][i]):
                    w1 = v
                    try:
                        w2 = df['gpt3_embed'][i][id + int(k)]
                    except IndexError:
                        continue
                    sim = cosine_similarity(w1, w2)
                    temp.append(sim) # a list of similarity scores for that response
                    #print('flag: ', temp)
                df[cur][i] = np.average(temp) # add other stats here
    df.to_csv(result + str(temp_file) + '.csv')
df.head()

Coherence k  2


/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_2423/4014079027.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cur][i] = np.average(temp)


Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10
Coherence k  2
Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10
Coherence k  2
Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10
Coherence k  2
Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10
Coherence k  2
Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10
Coherence k  2
Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10
Coherence k  2
Coherence k  3
Coherence k  4
Coherence k  5
Coherence k  6
Coherence k  7
Coherence k  8
Coherence k  9
Coherence k  10


,grid,content,n_words,gpt3_embed,gpt3_word_mv5,gpt3_word_mv10,gpt3_word_k2,gpt3_word_k3,gpt3_word_k4,gpt3_word_k5,gpt3_word_k6,gpt3_word_k7,gpt3_word_k8,gpt3_word_k9,gpt3_word_k10
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[0.009009403176605701, -0.017146145924925804,...",0.776863,0.777622,0.782686,0.784116,0.781053,0.782103,0.780552,0.784993,0.778847,0.779661,0.777854
1,11689,We have been using that opportunity to do more...,159,"[[-0.00924572255462408, -0.014978389255702496,...",0.786322,0.783411,0.789297,0.78867,0.779366,0.784245,0.785857,0.786791,0.782155,0.786047,0.78308
2,12376,Alright . um I live in not especially cool Spr...,468,"[[0.010747547261416912, -0.01882629469037056, ...",0.784362,0.785444,0.789207,0.785062,0.785558,0.788664,0.78434,0.790977,0.785502,0.791028,0.783467
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[-0.012155817821621895, -0.022000038996338844...",NaN,NaN,0.791977,0.790015,0.789015,0.788958,0.78967,0.791126,0.789266,0.786787,0.788519
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[0.003884142730385065, -0.01114651095122099, ...",NaN,NaN,0.785416,0.795869,0.78943,0.786664,0.786584,0.788698,0.781715,0.785351,0.787932


# Clinical factor scores

In [17]:
tlc = pd.read_csv('/Users/yancong/Desktop/4 clinical/00 Project Files/crossdx_clin.csv', index_col=0)
tlc = tlc[['grid', 'SSDvHC', 'group', 'tlc_01povspeech', 'tlc_02povcontent',	'tlc_03pressure',	'tlc_04distract',
	'tlc_05tangent', 'tlc_06derail', 'tlc_07incoh',	'tlc_08illogic',	'tlc_09clang',	'tlc_10neologism',
    	'tlc_11wordapprox',	'tlc_12circum',	'tlc_13lossgoal',	'tlc_14persev',	'tlc_15echo',	'tlc_16block',
        	'tlc_17stilt',	'tlc_18selfref', 'tlc_3f_inefficient',	'tlc_3f_incoherent',	'tlc_3f_impexpress']]
tlc.head()

,grid,SSDvHC,group,tlc_01povspeech,tlc_02povcontent,tlc_03pressure,tlc_04distract,tlc_05tangent,tlc_06derail,tlc_07incoh,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
1,10308,1.0,SSD,0,1,0,0,0,0,0,...,0,0,1,0,0,2,0,-0.406404,-0.069358,-0.018896
2,10311,0.0,HC,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.713934,-0.463481,0.197262
3,10316,1.0,SSD,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,10455,0.0,HC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,10582,1.0,SSD,0,0,0,1,0,1,0,...,1,1,0,0,0,0,0,-0.059923,-0.107333,-0.458561


In [19]:
temp = -1
for df in dfs:
    temp += 1
    df = df[['grid',
       'gpt3_word_k2', 'gpt3_word_k3', 'gpt3_word_k4',
       'gpt3_word_k5', 'gpt3_word_k6', 'gpt3_word_k7', 'gpt3_word_k8',
       'gpt3_word_k9', 'gpt3_word_k10', 'gpt3_word_mv5', 'gpt3_word_mv10']]
    df['grid'] = df['grid'].astype(str)
    df = df.merge(tlc, on=['grid'])
    df.to_csv(result + str(temp) + '_GT.csv')
df.head()

/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_2423/1643671373.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grid'] = df['grid'].astype(str)


,grid,gpt3_word_k2,gpt3_word_k3,gpt3_word_k4,gpt3_word_k5,gpt3_word_k6,gpt3_word_k7,gpt3_word_k8,gpt3_word_k9,gpt3_word_k10,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,10455,0.782686,0.784116,0.781053,0.782103,0.780552,0.784993,0.778847,0.779661,0.777854,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
1,11689,0.789297,0.78867,0.779366,0.784245,0.785857,0.786791,0.782155,0.786047,0.78308,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
2,12376,0.789207,0.785062,0.785558,0.788664,0.78434,0.790977,0.785502,0.791028,0.783467,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
3,12630,0.791977,0.790015,0.789015,0.788958,0.78967,0.791126,0.789266,0.786787,0.788519,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,13493,0.785416,0.795869,0.78943,0.786664,0.786584,0.788698,0.781715,0.785351,0.787932,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
